In [7]:
import sqlite3 as sq
import os 

In [4]:
os.chdir(r'D:\ML\Tutorials\Data Bases\SQLite')

con = sq.connect('saper.db') # if there is no connection, it will be created

# For accessing DB and making queries we have to create an object: cursor 
cur = con.cursor() # Creates an instance of Cursor class (i.e. instance has API)

# Connecting must be always closed
con.close()

Above methods isn't save. Use the context manager!

### Table Creation and Deletion

In [21]:
with sq.connect('saper.db') as con:
    cur = con.cursor()
    
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS users(
            name TEXT,
            sex INTEGER,
            age INTEGER,
            score INTEGER )
    
        """) 
    
# For Deletion: cur.execute(""" DROP TABLE users """)

### Some Rules
- SQL Commands: **capital letters;**
- Field: **lowercase letters;**

- For Creation Provide: ```IF NOT EXISTS```
- For Delition Provide: ```IF EXISTS```

In [47]:
# It is not convenient to wtite context manager each time.
# Let's create the func

def execute_query(query, con_name, is_table = True, n_entries = 1):
    """
    is_table: Only Table commands (INSERT, DROP, UPDATE...)
    n_entries: Number of entries to be returned
    
    """
    with sq.connect(con_name) as con:
        cur = con.cursor()
        if is_table:
            cur.execute(query)  
        else:
            cur.execute(query)
            res = cur.fetchmany(n_entries)
            return res

### Fields Limiters
Can define limits for fields. They are:
- ```IF EXISTS```
- ```IF NOT EXISTS```
- ```NOT NULL```
- ```DEFAULT value```
- ```PRIMARY KEY```
- ```AUTOINCREMENT```

In [29]:
con_name = 'saper.db'
qr = """ DROP TABLE IF EXISTS users """
execute_query(qr, con_name)

In [30]:
qr = """

CREATE TABLE IF NOT EXISTS users(
   user_id INTEGER PRIMARY KEY AUTOINCREMENT,
   name TEXT NOT NULL,
   sex TEXT DEFAULT 'Male',
   age INTEGER DEFAULT 24,
   score INTEGER )
   
"""

execute_query(qr, con_name)

### Inserting Values Into a Table
There are several options for inserting values into a table:
- ```INSERT INTO table_name (field_name_1...) VALUES (val_1...) or subquery```
- ```INSERT INTO table_name VALUES (val_1...) or subquery```

In [39]:
query = """

INSERT INTO users (name, sex, age, score) VALUES
    ('Milisa', 'Female', 19, 340),
    ('Stive','Male', 24, 1340),
    ('Kate', 'Female', 14, 2340),
    ('Tim', 'Male', 29, 3340)

"""

execute_query(query, con_name)

However, it isn't obligatory to provide default values. They can be ommited. But for multiple inserting we have to provide all values

### Simple SELECT
After a query has been executed, it returns a cursor object that has the following methods:
- ```cur.fetchall()``` - returns all entries ( not a good idea to use because problems with memory may appear);
- ```cur.fetchmany(size)``` - returns only provided number of entris;
- ```cur.fetchone()``` - returns only one entry;

In [48]:
query = """

SELECT name, sex, age
FROM users 
WHERE age < 20 and sex == 'Male' and score BETWEEN 300 and 500
ORDER BY score DESC
LIMIT 1

"""

execute_query(query, con_name, is_table = False)

[('Clark', 'Male', 18)]

### Values Update and Deletion


- ```UPDATE table_name SET (field_name_1...) = (value_1...) WHERE condition or subquery```
- ```DELETE FROM table_name WHERE condition or subquery``` 

Single Entry Update

In [49]:
query = """
UPDATE users SET (name, age) = ('Vladd', 18) WHERE user_id == 1

"""
execute_query(query, con_name, is_table = True)

Multiple Entries Update

In [51]:
# Have to change the function a bit
def execute_query(query, con_name, records, is_table = True, n_entries = 1):
    """
    is_table: Only Table commands (INSERT, DROP, UPDATE...)
    n_entries: Number of entries to be returned
    
    """
    with sq.connect(con_name) as con:
        cur = con.cursor()
        if is_table:
            cur.executemany(query, records)  
        else:
            cur.execute(query)
            res = cur.fetchmany(n_entries)
            return res

In [54]:
query = """

UPDATE users SET (name, sex, age) = (?, ?, ?) WHERE user_id = ?

"""
records = [('Vlad', 'Male', 24, 1), ('Oleg', 'Male', 18, 2), ('Tifony', 'Female', 24, 8)]
execute_query(query, con_name, records)

### Entry Deletion

In [57]:
query = """

DELETE FROM users WHERE user_id = ?

"""
user_ids = [(1,),(3,),(6,)]
execute_query(query, con_name, user_ids)

### Methods sqlite3 or API

- ```con.commit()``` - saves all changes into a database;
- ```con.close()``` - closes the current connection;
- ```con.rollback()``` - rolls back all changes if an error occurs;
- ```con.row_factory = sq.Row``` - results of a query will be returning Row object ( which is a dict );
- ```con.iterdump()``` - returns a list of SQL queries of how the current DB was created ( can be used to restore a DB )

- ```cur.execute(query)```
- ```cur.executemany(query)```
- ```cur.executescript(query)``` - executes a query like a SQL query;
- ```cur.lastrowid``` - returns an id of the last entry/record;
- ```cur.fetchall()``` - returns all results of a query as an ordered list;
- ```cur.fetchmany(size)``` - returns a results of a query according to the provided size;
- ```cur.fetchone()``` - returns only one result of a query;

- ```img = sq.Binary(img)``` - transforms an image into a special SQlite Object; 
- ```con = sq.connect( ':memory:' )``` - creates a temp DB into RAM